In [ ]:
csv_folder_path = './carbon_source'

In [93]:
def smooth(a,WSZ=5):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)

    start = np.cumsum(a[:WSZ-1])[::2]/r
    #print start,out0
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

def digitalOnly(s):
    import string
    al=string.maketrans('','')
    nodigs=al.translate(al, string.digits)
    return s.translate(al, nodigs)

def getLinearRegressionCoef(X,y):
    import numpy as np
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    reg.score(X, y)    
    return reg.coef_


def computeDerivetive_od(t, sOD, wsz = 5):
    from numpy import isnan
    coef_OD = np.zeros(len(sOD))
    t_mean = np.zeros(len(sOD))
    j = 0
    nan_found = False
    for i in range(len(sOD)):
        t_wsz = t[i-wsz/2:i+wsz/2+1]        
        sOD_wsz = np.expand_dims(np.log(sOD[i-wsz/2:i+wsz/2+1]),axis=1)    
        if len(sOD_wsz)!= wsz:
            coef_OD[i] = 0
        else:                
            if len(t_wsz)==len(sOD_wsz):                
                where_are_NaNs = isnan(sOD_wsz)
                if any(where_are_NaNs):
                    sOD_wsz[where_are_NaNs] = np.mean(sOD_wsz[~where_are_NaNs])
                    print "nan Found in array, replaced by mean value of the rest",i
                coef_OD[i]=getLinearRegressionCoef(t_wsz,sOD_wsz)
                t_mean[i] = np.mean(t_wsz)
    
    dt = t_mean[wsz/2:len(t_mean)-wsz/2]
    dt_pad = np.pad(dt,[wsz/2,wsz/2],'reflect',reflect_type='odd')
    
    return coef_OD, dt_pad


def draw_tangent(x1,y1,ind,e=0.1,plot=False):
    from scipy import interpolate
    import numpy as np
    import matplotlib.pyplot as plt
    x = np.squeeze(x1)
    y = np.squeeze(y1)
    # interpolate the data with a spline
    spl = interpolate.splrep(x,y)
    a = x[ind]
    fa = interpolate.splev(a,spl,der=0)     # f(a)
    fprime = interpolate.splev(a,spl,der=1) # f'(a)
    left = 0
    for i in range(ind,0,-1):        
        a_i = x[i]
        y_i = y[i]        
        tan_i = fa+fprime*(a_i-a) # tangent                
        if abs(tan_i-y_i) > abs(y[ind]*e):
            left = i
            break    
    right = len(x) - 1
    for i in range(ind+1,len(x)):        
        a_i = x[i]
        y_i = y[i]
        tan_i = fa+fprime*(a_i-a) # tangent        
        if abs(tan_i-y_i) > abs(y[ind]*e):
            right = i
            break
    small_t = np.linspace(x[left],x[right],10)
    tan = fa+fprime*(small_t-a) # tangent    
    if plot:
        plt.plot(a,fa,'om',small_t,tan,'--r')
        plt.plot(x,y)
    return left, right,small_t,tan


def getTimepoints_old(coef_OD,sOD_log, dt_pad,start_ind, thres=0.03, thres_range=0.8):
    coef_OD_fromStart_ind = coef_OD[start_ind:]
    ind = np.argmax(coef_OD_fromStart_ind) + start_ind        
    max_v = coef_OD[ind]
    
    time_max_coefod = dt_pad[ind]        
    right_side_exp = len(coef_OD)-1
    left_side_exp = 0
    for k in range(ind+1,len(coef_OD)):
        v_od = coef_OD[k]
        if (max_v-v_od)/max_v > 1-thres_range:
            right_side_exp = k
            break

    for k in reversed(range(ind)):
        v_od = coef_OD[k]
        if (max_v-v_od)/max_v > 1-thres_range:
            left_side_exp = k
            break

    time_left = dt_pad[left_side_exp]
    time_right = dt_pad[right_side_exp]  
    
    return time_max_coefod,time_left,time_right,left_side_exp,right_side_exp

def getTimepoints(coef_OD,sOD_log,dt_pad,start_ind):
    coef_OD_fromStart_ind = coef_OD[start_ind:]
    ind = np.argmax(coef_OD_fromStart_ind) + start_ind        
    max_v = coef_OD[ind]
    time_max_coefod = dt_pad[ind]        
    left_side_exp, right_side_exp,small_t,tan = draw_tangent(dt_pad,sOD_log,ind)
    time_left = dt_pad[left_side_exp]
    time_right = dt_pad[right_side_exp]  
    return time_max_coefod,time_left,time_right,left_side_exp,right_side_exp,small_t,tan, ind


def getPostiveStart(sOD):
    start_positive = 0
    for o in range(start_positive, len(sOD)):
        if sOD[o] <= 0:
            start_positive = o + 1
    if start_positive >= len(sOD):
        start_positive = -1
    return start_positive

def mkdir_p(path):
    import errno    
    import os
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            
            
def getStationaryTime(sOD):
    sOD = np.asarray(sOD)
    sta_index = len(sOD)-1
    last_value = sOD[sta_index]
    for i in range(len(sOD)-2,0,-1):
        if (abs(last_value-sOD[i])/last_value)>0.05:
            sta_index = i
            #print "stationary phase found"
            break
    return sta_index

def mkdir_p(path):
    import errno    
    import os
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [105]:
import os
import pandas as pd

def getfiles(folderPath):
    pths = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and 'stats' not in fn:
                pth = os.path.join(root, fn)
                pths.append(pth)
    return pths

pths = getfiles(csv_folder_path)

for path in pths:
    folder = path.split('.')[0]
    mkdir_p(folder)
    # pay attention of the csv format, if sep = ',' or ";"
    data = pd.read_csv(path,sep=';')    
    
    import numpy as np
    import matplotlib.pyplot as plt
    import os
    time = data.iloc[:,0]
    m_od = data.iloc[0:,1:]
    strains = list(m_od.columns)
    t = time.values

    t_k = (t[1:len(t)] - t[0:len(t)-1])/3600
    t_k = np.cumsum(t_k)
    t_k = np.insert(t_k,0,0)
    t_k = t_k.reshape(-1,1)
    t = t_k

    blank_od = 0.04
    pre_max_v = 0.7

    growth_rate_dict = dict()
    lag_phase_dict = dict()
    yield_dict = dict()
    for name in strains:
        od = m_od[name].values
        od_array = od - np.min(blank_od)
        od_array = np.squeeze(np.asarray(od_array))
        sOD = smooth(od_array)    
        
        start_positive = getPostiveStart(sOD)
        t = t_k
        if start_positive > 0:
            sOD = sOD[start_positive:]
            od = od[start_positive:]
            t = t_k[start_positive:]
            print name,'found negative od ', start_positive
        
        
        sOD_log = np.log(sOD)            
        coef_OD, dt_pad = computeDerivetive_od(t, sOD)
        
        #try:
        #    coef_OD, dt_pad = computeDerivetive_od(t, sOD)
        #except:
        #    continue
            
        ind_max_dODdt = np.argmax(coef_OD)
        OD_exp = sOD[ind_max_dODdt]
        time_max_dODdt = dt_pad[ind_max_dODdt]
  
        lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/pre_max_v
        m = t < lag_phase_pre
        start_ind = np.sum(m)

        time_max_od,time_left, time_right,left_side_exp,right_side_exp,small_t,tan,ind_max = getTimepoints(coef_OD,sOD_log,dt_pad,start_ind)

        time_max_dODdt = dt_pad[ind_max_dODdt]

        #od_max_coef = coef_OD[ind_max_dODdt]

        #ind = np.argmax(coef_OD)        
        max_v = coef_OD[ind_max]  
        #print "diffrence from estimate and real : ", ind, ind_max
        growth_rate = max_v

        
        
        # Update lag_phase using growth rate in an estimated window
        lag_phase_pre = time_max_dODdt - np.log(OD_exp/sOD[start_positive])/max_v
        lag_phase = lag_phase_pre

        sta_index = getStationaryTime(sOD)
        t_sta = dt_pad[sta_index]

        yield_mean = np.mean(sOD[sta_index:])
        print name+' yield'+"="+str(yield_mean)
        print name+' growth_rate'+"="+str(growth_rate)
        print "---------------------------------------"

        x = t
        y = sOD_log
        plt.plot(x,y,label='log(OD)')
        plt.plot(x,y,small_t,tan,'--r',label='log(OD)')
        plt.title(name)
        plt.xlabel("time")
        w = coef_OD
        plt.plot(x, w,label='dOD/dt/OD') # can c                
        plt.axvline(x=time_max_od,linestyle=':',color='orange')
        plt.legend()
        plt.savefig(os.path.join(folder,name +'-dODdtOD.png'))
        plt.close()   

        plt.plot(x,sOD,label='OD')    
        plt.axvline(x=time_left,linestyle=':',color='red')
        plt.axvline(x=time_max_od,linestyle=':',color='green')
        plt.axvline(x=t_sta,linestyle=':',color='yellow')
        plt.axvline(x=time_right,linestyle=':',color='red')
        plt.axvline(x=lag_phase,linestyle=':',color='purple')    
        plt.xlabel('time')
        plt.title(name)        
        plt.legend()
        plt.savefig(os.path.join(folder,name +'-OD.png'))
        plt.close()
        
        growth_rate_dict[name] = growth_rate
        lag_phase_dict[name] = lag_phase
        yield_dict[name] = yield_mean

    df = pd.DataFrame.from_dict([growth_rate_dict,lag_phase_dict, yield_dict])
    df = df.transpose()
    df2 = df.reindex(strains)
    df2.columns = ['Growth Rate', 'Lag Phase', 'yield']
    df2.to_csv(folder +'-stats.csv')
    

DLmalate10_R1 yield=0.2882315789473684
DLmalate10_R1 growth_rate=0.37578228851744067
---------------------------------------
DLmalate10_R2 yield=0.39273723577235775
DLmalate10_R2 growth_rate=0.6182038909813192
---------------------------------------
DLmalate10_R3 yield=0.2717381699346405
DLmalate10_R3 growth_rate=0.6718420560898659
---------------------------------------
DLmalate25_R1 yield=0.03336666666666666
DLmalate25_R1 growth_rate=1.5842455233445811
---------------------------------------
DLmalate25_R2 yield=0.04369533333333333
DLmalate25_R2 growth_rate=1.317224837793264
---------------------------------------
DLmalate25_R3 yield=0.05236933333333334
DLmalate25_R3 growth_rate=1.4114643548561727
---------------------------------------
DLmalate50_R1 yield=0.0032833333333333334
DLmalate50_R1 growth_rate=0.54223837233382
---------------------------------------
DLmalate50_R2 yield=0.005383333333333332
DLmalate50_R2 growth_rate=1.3105177235480132
---------------------------------------
D

Acetate200_R3 yield=0.055877777777777776
Acetate200_R3 growth_rate=0.4797269340183941
---------------------------------------
DAl10_R1 yield=0.29520777777777774
DAl10_R1 growth_rate=0.5578065073529842
---------------------------------------
DAl10_R2 yield=0.2587052380952381
DAl10_R2 growth_rate=0.5036105963798659
---------------------------------------
DAl10_R3 yield=0.31004000000000004
DAl10_R3 growth_rate=0.5258672488673526
---------------------------------------
DAl100_R1 yield=0.3067026666666667
DAl100_R1 growth_rate=0.35532412290284215
---------------------------------------
DAl100_R2 yield=0.31589222222222224
DAl100_R2 growth_rate=0.44896405159571534
---------------------------------------
DAl100_R3 yield=0.34530000000000005
DAl100_R3 growth_rate=0.35475307482210344
---------------------------------------
DAl200_R1 yield=0.18719904761904763
DAl200_R1 growth_rate=0.30550719754731115
---------------------------------------
DAl200_R2 yield=0.16935
DAl200_R2 growth_rate=0.54924463324

glc30uL_R3 yield=0.3323009523809524
glc30uL_R3 growth_rate=1.7795589063161188
---------------------------------------
glc30uL_R4 yield=0.4156793333333334
glc30uL_R4 growth_rate=0.9649745803471245
---------------------------------------
glc30uL_R5 found negative od  21
glc30uL_R5 yield=0.3197933333333334
glc30uL_R5 growth_rate=2.188920883696878
---------------------------------------
glc40uL_R1 found negative od  16
glc40uL_R1 yield=0.4515870370370371
glc40uL_R1 growth_rate=1.5719188083411253
---------------------------------------
glc40uL_R2 yield=0.3916882352941176
glc40uL_R2 growth_rate=1.1810820927239938
---------------------------------------
glc40uL_R3 found negative od  13
glc40uL_R3 yield=0.3681633333333334
glc40uL_R3 growth_rate=1.6216337161972092
---------------------------------------
glc40uL_R4 yield=0.39650428571428575
glc40uL_R4 growth_rate=1.1776850768240752
---------------------------------------
glc40uL_R5 found negative od  16
glc40uL_R5 yield=0.35352245614035094
glc40

pyruvate200_R3 yield=0.16074499999999997
pyruvate200_R3 growth_rate=0.4271165687755039
---------------------------------------
Ribose10_R1 yield=0.34074444444444446
Ribose10_R1 growth_rate=0.4263062517648974
---------------------------------------
Ribose10_R2 yield=0.3471369696969696
Ribose10_R2 growth_rate=0.5641129693984318
---------------------------------------
Ribose10_R3 yield=0.3468880952380952
Ribose10_R3 growth_rate=0.45457703398874927
---------------------------------------
Ribose100_R1 yield=0.5597005333333332
Ribose100_R1 growth_rate=0.5033036192240455
---------------------------------------
Ribose100_R2 yield=0.5612058974358974
Ribose100_R2 growth_rate=0.5358932155090103
---------------------------------------
Ribose100_R3 yield=0.5669485333333333
Ribose100_R3 growth_rate=0.4713838118229243
---------------------------------------
Ribose200_R1 yield=0.6507954166666666
Ribose200_R1 growth_rate=0.48127880013846314
---------------------------------------
Ribose200_R2 yield=0.6

pro10_R3 yield=0.03849533333333334
pro10_R3 growth_rate=0.2680881486995313
---------------------------------------
pro100_R1 yield=0.027659629629629628
pro100_R1 growth_rate=0.21248046138500995
---------------------------------------
pro100_R2 yield=0.026131515151515152
pro100_R2 growth_rate=0.26140674084564286
---------------------------------------
pro100_R3 yield=0.028797499999999997
pro100_R3 growth_rate=0.326151277574973
---------------------------------------
pro200_R1 yield=0.03726866666666667
pro200_R1 growth_rate=0.8375565773154757
---------------------------------------
pro200_R2 yield=0.043609999999999996
pro200_R2 growth_rate=0.41445606088397663
---------------------------------------
pro200_R3 yield=0.043911851851851855
pro200_R3 growth_rate=0.41804950206406044
---------------------------------------
tre0_R1 yield=0.5727866666666667
tre0_R1 growth_rate=0.4459057517359965
---------------------------------------
tre0_R2 yield=0.5617090909090908
tre0_R2 growth_rate=0.53191765

Tartaric10_R3 yield=0.033075238095238095
Tartaric10_R3 growth_rate=0.39827617010167404
---------------------------------------
Tartaric25_R1 yield=0.0346516049382716
Tartaric25_R1 growth_rate=0.29555722301552695
---------------------------------------
Tartaric25_R2 yield=0.03346238095238095
Tartaric25_R2 growth_rate=0.3545954816193106
---------------------------------------
Tartaric25_R3 yield=0.03251843137254903
Tartaric25_R3 growth_rate=0.3203032602400116
---------------------------------------
Tartaric5_R1 yield=0.03404233333333333
Tartaric5_R1 growth_rate=0.38643455874663907
---------------------------------------
Tartaric5_R2 yield=0.030956318407960197
Tartaric5_R2 growth_rate=0.5542571732121285
---------------------------------------
Tartaric5_R3 yield=0.029739285714285708
Tartaric5_R3 growth_rate=0.543575621884244
---------------------------------------
Deoxyadenosine1.2_R1 yield=0.02870820512820513
Deoxyadenosine1.2_R1 growth_rate=0.24442578130983653
---------------------------

asparagine10_R2 yield=0.04559047619047619
asparagine10_R2 growth_rate=0.4182456841282598
---------------------------------------
asparagine10_R3 yield=0.04456190476190476
asparagine10_R3 growth_rate=0.2553580146585244
---------------------------------------
asparagine25_R1 yield=0.08163666666666666
asparagine25_R1 growth_rate=0.4015747353214178
---------------------------------------
asparagine25_R2 yield=0.110835
asparagine25_R2 growth_rate=0.5451322702562024
---------------------------------------
asparagine25_R3 yield=0.12924000000000002
asparagine25_R3 growth_rate=0.34047947332452194
---------------------------------------
asparagine5_R1 yield=0.031911794871794866
asparagine5_R1 growth_rate=0.5424190317636696
---------------------------------------
asparagine5_R2 yield=0.03197407407407407
asparagine5_R2 growth_rate=0.5068996994756356
---------------------------------------
asparagine5_R3 yield=0.03304190476190476
asparagine5_R3 growth_rate=0.4126638828990827
-----------------------

# For the name with _R1 _R2 _R3

In [199]:
import os
import pandas as pd
from itertools import groupby 

def getfiles(folderPath):
    pths = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and 'stats' in fn:
                pth = os.path.join(root, fn)
                pths.append(pth)
    return pths

csv_folder_path = './csv'
pths = getfiles(csv_folder_path)

extrac_gr = "Growth Rate"
extrac_yi = "yield"

yield_list = []
growth_rate_list = []
name_list = []
for path in pths:
    print path
    data = pd.read_csv(path)
    data2 = data.set_index(data.columns[0])
    index = data.iloc[:,0]
    # initializing list  
    test_list = list(index)
    # sort list  
    # essential for grouping 
    test_list.sort() 
    # printing the original list  
    # using lambda + itertools.groupby() + split() 
    # group similar substrings 
    group_ind = [list(i) for j, i in groupby(test_list, 
                      lambda a: a.split('_')[0])]    
    
    for g in group_ind:
        #print g
        yield_dict = dict()
        growth_dict = dict()
        i = 1
        for strain in g:
            init = 'R'+str(i)
            yield_dict[init] = data2.loc[strain,extrac_yi]
            growth_dict[init] = data2.loc[strain, extrac_gr]
            i = i +1
        yield_list.append(yield_dict)
        growth_rate_list.append(growth_dict)
        name_list.append(g[0].split('_')[0])



./csv/MG1655 10082019 N-stats.csv
./csv/MG1655 25062019-stats.csv
./csv/MG1655 22062019-stats.csv
./csv/MG1655 14082019 N-stats.csv
./csv/MG1655 20082019 N-stats.csv
./csv/MG1655 23062019-stats.csv
./csv/MG1655 17082019 N-stats.csv
./csv/MG1655 13082019 N-stats.csv
./csv/MG1655 24062019-stats.csv
./csv/MG1655 19062019-stats.csv
./csv/MG1655 07072019-stats.csv
./csv/MG1655 29062019-stats.csv
./csv/MG1655 03082019-stats.csv
./csv/MG1655 14062019-stats.csv
./csv/MG1655 26062019-stats.csv
./csv/MG1655 08072019-stats.csv
./csv/MG1655 28062019-stats.csv
./csv/MG1655 02082019-stats.csv
./csv/MG1655 27062019-stats.csv
./csv/MG1655 05082019-stats.csv


In [200]:
fn = os.path.split(csv_folder_path)[1]

df = pd.DataFrame(yield_list) 
df = df.set_index([pd.Index(name_list)])
df
df.to_csv('./results/yield_all_strains_'+fn+'.csv')

df = pd.DataFrame(growth_rate_list) 
df = df.set_index([pd.Index(name_list)])
df
df.to_csv('./results/growth_rate_all_strains_'+fn+'.csv')

# Load csv for the name without _R1 _R2 _R3

In [106]:
import os
import pandas as pd

def getfiles(folderPath):
    pths = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and 'stats' in fn:
                pth = os.path.join(root, fn)
                pths.append(pth)
    return pths

csv_folder_path = './carbon_source'
pths = getfiles(csv_folder_path)

extrac_gr = "Growth Rate"
extrac_yi = "yield"

yield_list = []
growth_rate_list = []
name_list = []
for path in pths:
    print path
    data = pd.read_csv(path)
    data2 = data.set_index(data.columns[0])
    pre_strain ='no'
    for strain in data2.index:
        if pre_strain not in strain:
            try:
                if pre_strain != 'no':
                    print "s3",pre_strain
                    yield_list.append(yield_dict)
                    growth_rate_list.append(growth_dict)
                    name_list.append(pre_strain)
            except:
                print pre_strain,strain

        if pre_strain != strain:
            if pre_strain not in strain:
                pre_strain = strain
                #name_list.append(pre_strain)
                #print "s1",pre_strain, strain
                i = 1
                yield_dict = dict()
                growth_dict = dict()

        init = 'R'+str(i)
        if pre_strain in strain:
            #print "s2",pre_strain, strain
            #print init,data2.loc[strain,extrac]
            yield_dict[init] = data2.loc[strain,extrac_yi]
            growth_dict[init] = data2.loc[strain, extrac_gr]
            i = i +1


print "last s3",pre_strain
yield_list.append(yield_dict)
growth_rate_list.append(growth_dict)
name_list.append(pre_strain)


./carbon_source/Glc v Carbon sources 6 version 2-stats.csv
s3 DLmalate10_R1
s3 DLmalate10_R2
s3 DLmalate10_R3
s3 DLmalate25_R1
s3 DLmalate25_R2
s3 DLmalate25_R3
s3 DLmalate50_R1
s3 DLmalate50_R2
s3 DLmalate50_R3
s3 glc0.4_R1
s3 glc0.4_R2
s3 glc0.4_R3
s3 glucuronica10_R1
s3 glucuronica10_R2
s3 glucuronica10_R3
s3 glucuronica25_R1
s3 glucuronica25_R2
s3 glucuronica25_R3
s3 glucuronica50_R1
s3 glucuronica50_R2
s3 glucuronica50_R3
s3 ketobutyrica1_R1
s3 ketobutyrica1_R2
s3 ketobutyrica1_R3
s3 ketobutyrica2_R1
s3 ketobutyrica2_R2
s3 ketobutyrica2_R3
s3 ketobutyrica5_R1
s3 ketobutyrica5_R2
s3 ketobutyrica5_R3
s3 Llactate10_R1
s3 Llactate10_R2
s3 Llactate10_R3
s3 Llactate25_R1
s3 Llactate25_R2
s3 Llactate25_R3
s3 Llactate5_R1
s3 Llactate5_R2
s3 Llactate5_R3
s3 methylpyr10_R1
s3 methylpyr10_R2
s3 methylpyr10_R3
s3 methylpyr25_R1
s3 methylpyr25_R2
s3 methylpyr25_R3
s3 methylpyr5_R1
s3 methylpyr5_R2
s3 methylpyr5_R3
s3 Saccharicac0.9_R1
s3 Saccharicac0.9_R2
s3 Saccharicac0.9_R3
s3 Saccharicac18_

In [112]:
data2.index

Index([u'glc10uL_R1', u'glc10uL_R2', u'glc10uL_R3', u'glc10uL_R4',
       u'glc10uL_R5', u'glc100uL_R1', u'glc100uL_R2', u'glc100uL_R3',
       u'glc100uL_R4', u'glc100uL_R5', u'glc20uL_R1', u'glc20uL_R2',
       u'glc20uL_R3', u'glc20uL_R4', u'glc20uL_R5', u'glc30uL_R1',
       u'glc30uL_R2', u'glc30uL_R3', u'glc30uL_R4', u'glc30uL_R5',
       u'glc40uL_R1', u'glc40uL_R2', u'glc40uL_R3', u'glc40uL_R4',
       u'glc40uL_R5', u'glc50uL_R1', u'glc50uL_R2', u'glc50uL_R3',
       u'glc50uL_R4', u'glc50uL_R5', u'glc60uL_R1', u'glc60uL_R2',
       u'glc60uL_R3', u'glc60uL_R4', u'glc60uL_R5', u'glc70uL_R1',
       u'glc70uL_R2', u'glc70uL_R3', u'glc70uL_R4', u'glc70uL_R5',
       u'glc80uL_R1', u'glc80uL_R2', u'glc80uL_R3', u'glc80uL_R4',
       u'glc80uL_R5', u'glc90uL_R1', u'glc90uL_R2', u'glc90uL_R3',
       u'glc90uL_R4', u'glc90uL_R5'],
      dtype='object', name=u'Unnamed: 0')

In [151]:
fn = os.path.split(csv_folder_path)[1]

df = pd.DataFrame(yield_list) 
df = df.set_index([pd.Index(name_list)])
df
df.to_csv('./results/yield_all_strains_'+fn+'.csv')

df = pd.DataFrame(growth_rate_list) 
df = df.set_index([pd.Index(name_list)])
df
df.to_csv('./results/growth_rate_all_strains_'+fn+'.csv')